# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models_tf2 import Models
from botbidder import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


In [2]:
models.search_threshold = 0.07
models.double_dummy_calculator = True
# Filter to remove hands not consistent with the bidding. Used during play
models.bid_accept_play_threshold = 0.01
#sampler.use_distance = False
#sampler.bidding_threshold_sampling = 0.25


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
from ddsolver import ddsolver
dds = ddsolver.DDSolver()

hands = ["AJ.KJ9.A9764.AJ8","T853.AT65.Q8.T52", "K976.Q874.KT.K74", "Q42.32.J532.Q963"]
#hands = ['AKQJTx.AQTx.Ax.x', 'xxx.Kx.KT8x.KJxx', 'x.J98xxxx.9.QTxx', '98x..QJxxxx.A98x']

In [4]:
# North is dealer
dealer = 0
bidder_bots = [BotBid([False, True], hand, models, sampler, i, dealer, dds, False, False) for i, hand in enumerate(hands)]

In [5]:
auction = [] 

turn_i = 0 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    #if step== 5:
    #     bid.bid='X'
    #if step== 1:
    #     bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1D', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1D', 'insta_score': 1.199, 'alert': 'False', 'explanation': '3+ !D -- 6-!C; 3+!D; 6-!H; 6-!S; 11-21 HCP'}], 'hcp': [6.4, 8.1, 6.4], 'shape': [3.7, 3.3, 2.8, 3.2, 3.6, 3.4, 2.8, 3.1, 3.7, 3.2, 2.8, 3.2], 'explanation': '3+ !D -- 6-!C; 3+!D; 6-!H; 6-!S; 11-21 HCP'}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candidates': [{'call': 'PASS', 'insta_score': 1.2, 'alert': 'False', 'explanation': ' -- ; 14- HCP'}], 'hcp': [10.7, 9.6, 13.3], 'shape': [3.2, 3.2, 3.0, 3.5, 3.2, 3.2, 2.9, 3.6, 2.6, 2.6, 4.9, 2.9], 'explanation': ' -- ; 14- HCP'}
{'bid': '1H', 'who': 'NN', 'quality': '1', 'candidates': [{'call': '1H', 'insta_score': 1.2, 'alert': 'False', 'explanation': 'Bidable suit -- 4+!H; 6-!S; 6-29 HCP; Forcing'}], 'hcp': [9.1, 13.1, 6.3], 'shape': [3.3, 3.3, 2.8, 3.6, 2.6, 2.7, 4.9, 2.8, 3.1, 3.1, 3.2, 3.7], 'explanation': 'Bidable suit -- 4+!H; 6-!S; 6-29 HCP; Forcing'}
{'bid': 'PASS', 'who': 'NN', 'quality': '1', 'candida